<a href="https://colab.research.google.com/github/ykitaguchi77/Strabismus_AI_project/blob/main/5-fold%20crossvalidation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Strabismus 5-fold crossvalidation**

In [ ]:
"""
CSVを作成し、ファイル名-正解-予測を記載

データを5分割（division1~5)
set1: 1がtest、2~5がtrain
set2: 2がtest、1、3~5がtrain
set3:
set4: 
set5: 5がtest、1~4がtrain
それぞれのdivisionに、exoとcontのフォルダを置いておく

各セットについてトレーニング→CSVに予測の結果を記載していく
全てのデータが揃ったら、正解-予測のデータから、正解率、感度、特異度、ROC curveの計算を行う
"""

#**Split dataset for Crossvalidation**
trainセットを５分割、うち1つをvalセット、残りの合計をtestセットに分割

In [1]:
import random
import glob
import os
import shutil
import numpy as np
import time
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')


'''
-----orig_data-----grav
                |--cont
↓
↓

-----dst_data[0]------dst_train[0]----grav
  |                |               |-- cont
  |                |--dst_val[0]------grav
  |                                |--cont
  |
  |--dst_data[1]------dst_train[1]----grav
  |                |               |-- cont
  |                |--dst_val[1]------grav
  |                                |--cont
  ...
  |--dst_data[1]------dst_train[9]----grav
                   |               |-- cont
                   |--dst_val[9]------grav
                                   |--cont
'''

Mounted at /content/drive


'\n-----orig_data-----grav\n                |--cont\n↓\n↓\n\n-----dst_data[0]------dst_train[0]----grav\n  |                |               |-- cont\n  |                |--dst_val[0]------grav\n  |                                |--cont\n  |\n  |--dst_data[1]------dst_train[1]----grav\n  |                |               |-- cont\n  |                |--dst_val[1]------grav\n  |                                |--cont\n  ...\n  |--dst_data[1]------dst_train[9]----grav\n                   |               |-- cont\n                   |--dst_val[9]------grav\n                                   |--cont\n'

In [2]:
orig_path = "/content/drive/MyDrive/Deep_learning/Strabismus/Dataset_250px_20211111"
dst_path = "/content/drive/MyDrive/Deep_learning/Strabismus/Dataset_250px_crossvalidation"  # フォルダ名
split_num = 5  #データをいくつに分割するかを記載

In [3]:
def get_path(orig_path, dst_path, split_num):
    classes = os.listdir(orig_path) #クラス名を取得
    #データの分割数を設定
    data_list = [0]*len(classes)
    k=0
    for i in classes:
        data_list[k] = glob.glob(orig_path+'/'+i+'/*')
        k+=1
    split_length = int(len(data_list)/split_num)
    return data_list, classes, split_length

def makefolder(orig_path, dst_path, classes):
    #フォルダを作成
    if not os.path.exists(dst_path):  # ディレクトリがなかったら
        os.mkdir(dst_path)  # 作成したいフォルダ名を作成
        for i in range(split_num):
            os.mkdir(dst_path+'/'+str(i))
            os.mkdir(dst_path+'/'+str(i)+'/train')
            os.mkdir(dst_path+'/'+str(i)+'/val')
            for j in classes:
                os.mkdir(dst_path+'/'+str(i)+'/train/'+j)
                os.mkdir(dst_path+'/'+str(i)+'/val/'+j)

def split_data_list(data_list, split_num):
    split_data, dst_data, dst_train, dst_val, dst_test = [0]*split_num, [0]*split_num, [0]*split_num, [0]*split_num, [0]*split_num

    #データの分割
    split_data = list(np.array_split(data_list, split_num))

    #データセット全体と分割したデータの差分を取り、dst_dataに格納

    dst_data = [0] * split_num
    for i in range(split_num):
        dst_data[i] = [x for x in data_list if x not in split_data[i]]

    #トレーニングセット、バリデーションセット、テストセットのリスト作成
    for i in range(split_num):
        dst_train[i] = dst_data[i]
        dst_val[i] = split_data[i]  #テストセット
    
    return dst_train, dst_val

def copy_to_folders(split_num, class_name, dst_train, dst_val, dst_path):
    k=0
    for i in range(split_num):
        dst_path_train = dst_path +'/'+str(i)+'/train/'+class_name
        dst_path_val = dst_path +'/'+str(i)+'/val/'+class_name
        for p in dst_train[k]:  # 選択したファイルを目的フォルダにコピー
            shutil.copy(p, dst_path_train)
            #print(p)
            print(dst_path_train)

        for p in dst_val[k]:  # 選択したファイルを目的フォルダにコピー
            shutil.copy(p, dst_path_val)
            #print(p)    
            print(dst_path_val)

        k+=1

In [ ]:
#Dataの５分割
data_list, classes, split_length = get_path(orig_path, dst_path, split_num)
makefolder(orig_path, dst_path, classes)


print(classes)
k=0
for i in range(len(classes)):
    dst_train, dst_val = split_data_list(data_list[k], split_num)
    print(classes[k])
    copy_to_folders(split_num, classes[k], dst_train, dst_val, dst_path)
    k+=1

In [28]:
classes = os.listdir(dst_path+"/0/val/")
split_num = 5
img_num = len(glob.glob(dst_path+"/*/val/*/*"))
#print(glob.glob(dst_path+"/*/val/*/*"))
#print(img_num)

#Make CSV file
cols = ["num", "division", "class", "prediction"]
df_cross = pd.DataFrame(index=list(range(img_num)), columns=cols)
print(df_cross)

t=0
for i in range(split_num):
    for j in classes:
        print(str(i) + ", "+str(j))
        img_list = os.listdir(dst_path+"/"+str(i)+"/val/"+str(j))
        print(img_list)
        for k in img_list:
            df_cross.iloc[t,0]=str(k)
            df_cross.iloc[t,1]=str(i)
            df_cross.iloc[t,2]=str(j)
            t+=1

            
df_cross


      num division class prediction
0     NaN      NaN   NaN        NaN
1     NaN      NaN   NaN        NaN
2     NaN      NaN   NaN        NaN
3     NaN      NaN   NaN        NaN
4     NaN      NaN   NaN        NaN
...   ...      ...   ...        ...
1025  NaN      NaN   NaN        NaN
1026  NaN      NaN   NaN        NaN
1027  NaN      NaN   NaN        NaN
1028  NaN      NaN   NaN        NaN
1029  NaN      NaN   NaN        NaN

[1030 rows x 4 columns]
0, cont
['6868.jpg', '1912.jpg', '7111.jpg', '5169.jpg', '3092.jpg', '4796.jpg', '9256.jpg', '6738.jpg', '8739.jpg', '8728.jpg', '9595.jpg', '1017.jpg', '4163.jpg', '2102.jpg', '6354.jpg', '1193.jpg', '2506.jpg', '1574.jpg', '8134.jpg', '5623.jpg', '2063.jpg', '5539.jpg', '3153.jpg', '4173.jpg', '2298.jpg', '1295.jpg', '1828.jpg', '1459.jpg', '5889.jpg', '8015.jpg', '2014.jpg', '8010.jpg', '5977.jpg', '137.jpg', '8102.jpg', '1393.jpg', '2575.jpg', '1319.jpg', '5782.jpg', '4818.jpg', '2031.jpg', '3139.jpg', '7963.jpg', '1025.jpg', '7400.j

,num,division,class,prediction
0,6868.jpg,0,cont,NaN
1,1912.jpg,0,cont,NaN
2,7111.jpg,0,cont,NaN
3,5169.jpg,0,cont,NaN
4,3092.jpg,0,cont,NaN
...,...,...,...,...
1025,82.jpg,4,exo,NaN
1026,929.jpg,4,exo,NaN
1027,994.jpg,4,exo,NaN
1028,715.jpg,4,exo,NaN


#**作ったフォルダの削除**

In [ ]:
dst_path = "/content/drive/MyDrive/Deep_learning/Strabismus/Dataset_250px_crossvalidation"
directory = dst_path
try:
    shutil.rmtree(directory)
except FileNotFoundError:
    pass